# Single Model Energy-dependent Classification Performance Analysis

In [1]:
%matplotlib inline

# Simply Run the usual experiment, using -L option to specify the model to load
# Sets up the model and the data

%run -im CaloDNN.ClassificationExperiment -- --NoAnalysis --NoTrain --cpu -L TrainedModels.Test/CaloDNN_32_1_0.01_RMSprop_0.01_Merged.9

Using GPU 3
Found 12 CPUs and 4 GPUs. Using 3 threads. max_threads = 12
HyperParameter Scan:  240 possible combiniations.
______________________________________
ScanConfiguration
______________________________________
Picked combination:  0
Combo[0]={'Width': 32, 'Depth': 1, 'lr': 0.01, 'optimizer': "'RMSprop'", 'decay': 0.01}
Model Filename:  CaloDNN_32_1_0.01_RMSprop_0.01
______________________________________
Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 3: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5105)


Searching in : /data/LCD/*/*.h5
Found 639 files.
Train Class Index Map: {'Pi0': 0, 'ChPi': 1, 'Gamma': 2, 'Ele': 3}
Caching data on disk for faster processing after first epoch. Hope you have enough disk space.
Loading Model From: TrainedModels.Test/CaloDNN_32_1_0.01_RMSprop_0.01_Merged.9
Output Directory: TrainedModels/CaloDNN_32_1_0.01_RMSprop_0.01_Merged.9
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 25, 25, 25)    0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 5, 5, 60)      0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)             

In [ ]:
TrainSampleList,TestSampleList,Norms,shapes=SetupData(FileSearch,
                                                      ECAL,HCAL,True,NClasses,
                                                      [float(NSamples)/MaxEvents,
                                                       float(NTestSamples)/MaxEvents],
                                                      Particles,
                                                      BatchSize,
                                                      multiplier,
                                                      ECALShape,
                                                      HCALShape,
                                                      ECALNorm,
                                                      HCALNorm)

Test_genC = MakeGenerator(ECAL, HCAL, TestSampleList, NTestSamples, LCDNormalization(Norms),
                          Merge=False,
                          batchsize=BatchSize,
                          shapes=shapes,
                          n_threads=n_threads,
                          multiplier=multiplier,
                          cachefile="/tmp/CaloDNN-Analysis.h5")

print "Loading Data into Memory:"
Test_genC.PreloadData(n_threads_cache)
Test_X_ECAL, Test_X_HCAL, target, Test_Y = tuple(Test_genC.D)


Searching in : /data/LCD/*/*.h5
Found 639 files.
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [ ]:
# Make the ROC Curves
# Also performs inference on the test data, returning the results
from DLAnalysis.Classification import *
result,NewMetaData=MultiClassificationAnalysis(MyModel,[Test_X_ECAL,Test_X_HCAL],Test_Y,BatchSize,
                                                       IndexMap={0:'Pi0', 2:'ChPi', 3:'Gamma', 1:'Ele'})

/home/afarbin/.virtualenvs/keras2/local/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
# Bin the data
Energy=target[:,:,2].flatten()

def AUCvsEnergy(E_min=10.,E_max=510.,E_bins=100.):
    BD,E_binning=BinDataIndex(Energy, E_min, E_max, E_bins)
    # Run the Classification Analysis in Bins
    return BinMultiClassificationAnalysis(MyModel,Test_Y=Test_Y,Y_binning=E_binning,
                                          bin_indecies=BD, result=result,
                                          IndexMap={0:'Pi0', 2:'ChPi', 3:'Gamma', 1:'Ele'})

In [ ]:
# Full Energy Range
Res=AUCvsEnergy(10.,510.,50.)

In [ ]:
# 10 to 100 GeV
Res=AUCvsEnergy(10.,110.,20.)

In [ ]:
# 250 to 350 GeV
Res=AUCvsEnergy(250.,350.,50.)

In [ ]:
tmp=plt.hist(Energy,bins=100)